<a href="https://colab.research.google.com/github/Ramanarayanan/LLM-Finetuning/blob/main/Fine_tune_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from datasets import load_dataset

# Replace 'yelp_polarity' with your desired open-source dataset
dataset = load_dataset("yelp_polarity", split="train[:100]")  # Use only 200 examples for demonstration

def preprocess(example):
    return {
        "prompt": example["text"],
        "completion": "positive" if example["label"] == 1 else "negative"
    }

dataset = dataset.map(preprocess)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


In [ ]:
def tokenize(example):
    text = "### Prompt: " + example["prompt"] + "\n### Sentiment: " + example["completion"]
    result = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128
    )
    # For causal LM, labels should be the same as input_ids, or use -100 to mask out padding for loss computation
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora-tinyllama-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=1,  # increase for better results
    learning_rate=2e-4,
    save_strategy="epoch",
    logging_steps=10,
    push_to_hub=False,  # Set to False for now, will push after
    no_cuda=True        # Forces CPU usage
)
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-724433698.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: Expected input batch_size (128) to match target batch_size (1).

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

# PARAMETERS – change to your liking
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
HF_REPO = "RamAi24/tinyllama-lora-cpu-demo"  # change!
MAX_LEN = 128

# --- 1. Load and preprocess dataset (using a tiny sample for demonstration) ---
dataset = load_dataset("yelp_polarity", split="train[:200]")  # demo: 200 samples

def preprocess_fn(example):
    return {
        "prompt": example["text"],
        "completion": "positive" if example["label"] == 1 else "negative"
    }

dataset = dataset.map(preprocess_fn)

# --- 2. Tokenizer + Tokenization ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    texts = [
        f"### Prompt: {prompt}\n### Sentiment: {completion}"
        for prompt, completion in zip(batch["prompt"], batch["completion"])
    ]
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=MAX_LEN
    )
    # For language modeling, use labels = input_ids, mask with -100 for padding
    labels = []
    for ids in encodings["input_ids"]:
        labels.append([id if id != tokenizer.pad_token_id else -100 for id in ids])
    encodings["labels"] = labels
    return encodings

tokenized_dataset = dataset.map(tokenize_batch, batched=True, remove_columns=dataset.column_names)

# --- 3. Load model and apply LoRA ---
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# --- 4. Data collator (standard for language modeling) ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# --- 5. Training arguments (CPU ONLY) ---
training_args = TrainingArguments(
    output_dir="./lora-tinyllama-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    save_strategy="epoch",
    logging_steps=10,
    push_to_hub=False,
    no_cuda=True,      # ensure CPU training
    eval_steps=None
)

# --- 6. Trainer and training ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()




Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/tmp/ipython-input-1896394807.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.017000
20,3.081300
30,2.747200
40,2.753500
50,2.568800
60,2.647800
70,2.399400
80,2.304600
90,2.290900
100,2.549300


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-688b879e-0bbc7d95227d351e5fb2b01f;f7a27090-b544-4b4e-b062-32e7b29b9cca)

Invalid username or password.

In [ ]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `alltoken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential

In [ ]:
model.push_to_hub("RamAi24/tinyllama-lora-cpu-demo")
tokenizer.push_to_hub("RamAi24/tinyllama-lora-cpu-demo")


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pmkxec6r8/adapter_model.safetensors:  12%|#2        |  563kB / 4.52MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpmusjdm3k/tokenizer.model      : 100%|##########|  500kB /  500kB            

CommitInfo(commit_url='https://huggingface.co/RamAi24/tinyllama-lora-cpu-demo/commit/0639daeab55f4df99591a2c2f8d5230c0af2a4a5', commit_message='Upload tokenizer', commit_description='', oid='0639daeab55f4df99591a2c2f8d5230c0af2a4a5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RamAi24/tinyllama-lora-cpu-demo', endpoint='https://huggingface.co', repo_type='model', repo_id='RamAi24/tinyllama-lora-cpu-demo'), pr_revision=None, pr_num=None)